In [2]:
# Cell #: 1 Library Imports
import pandas as pd
import networkx as nx
import uunet.multinet as ml
import itertools
import numpy as np  
from math import log

In [3]:
#Cell #: 2 File Paths
# File paths for the Excel sheets
file_paths = [
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_1.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_2.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_3.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_4.xlsx",
    "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_5.xlsx"
]


In [4]:
# Cell #: 3 Network Triming Logic
# Reading the Excel files
cases_data = [pd.read_excel(file_path) for file_path in file_paths]

# Function to apply the filtering logic and track removed edges
def filter_edges_and_save_removed(cases):
    max_flow_per_edge = {}
    filtered_cases = []
    removed_edges = []

    # Step 1: Determine the maximum flow for each edge across all cases
    for case in cases:
        for _, row in case.iterrows():
            edge = (row['From_Node'], row['To_Node'])
            flow = row['Flow']
            max_flow_per_edge[edge] = max(max_flow_per_edge.get(edge, 0), flow)

    # Steps 2 and 3: Filtering edges, setting correct layer, and tracking removed edges
    for case_index, case in enumerate(cases, start=1):
        filtered_rows = []
        removed_edges_rows = []
        for _, row in case.iterrows():
            edge = (row['From_Node'], row['To_Node'])
            if row['Flow'] >= max_flow_per_edge[edge] * 0.9:
                row['From_Layer'] = case_index  # Set to correct case number
                row['To_Layer'] = case_index    # Set to correct case number
                filtered_rows.append(row)
            else:
                removed_edges_rows.append(row)
        filtered_cases.append(pd.DataFrame(filtered_rows))
        removed_edges.append(pd.DataFrame(removed_edges_rows))

    return filtered_cases, removed_edges

# Apply the filtering logic and get removed edges
filtered_cases_data, removed_edges_data = filter_edges_and_save_removed(cases_data)

# Print the results and optionally save them to files
for i, (filtered_case, removed_edges_case) in enumerate(zip(filtered_cases_data, removed_edges_data), 1):
    #print(f"Case {i} - Filtered Data:\n", filtered_case.head())
    print(f"Case {i} - Removed Edges:\n", removed_edges_case, "\n")
    # Optionally, save the filtered cases to files
    filtered_case.to_excel(f"/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Filtered_Case_{i}.xlsx", index=False)
    
    # Optionally, save the removed edges to files
    #removed_edges_case.to_excel(f"/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Removed_Edges_Case_{i}.xlsx", index=False)


Case 1 - Removed Edges:
      From_Node  From_Layer  To_Node  To_Layer        Flow
11         2.0         1.0     12.0       1.0   39.476633
16        59.0         1.0     70.0       1.0   65.473726
17        12.0         1.0     30.0       1.0   77.406378
18        44.0         1.0     30.0       1.0  102.065698
19        18.0         1.0     44.0       1.0  111.489735
..         ...         ...      ...       ...         ...
241      148.0         2.0    159.0       2.0  113.276302
242      159.0         2.0    161.0       2.0  113.276302
244      125.0         2.0    124.0       2.0  114.968040
246      124.0         2.0    136.0       2.0  129.942968
247      136.0         2.0    148.0       2.0  129.942968

[66 rows x 5 columns] 

Case 2 - Removed Edges:
      From_Node  From_Layer  To_Node  To_Layer       Flow
0          3.0         1.0     34.0       1.0   0.000000
1          3.0         1.0     34.0       1.0   0.000000
2          6.0         1.0     36.0       1.0   0.000000
3

In [5]:
import pandas as pd

# Assuming 'filtered_cases_data' is a list of DataFrames
for i, layer_data in enumerate(filtered_cases_data):
    print(f"Layer {i+1} Data:")
    print(layer_data.head())  # Prints the first 5 rows of each DataFrame
    print("\n")


Layer 1 Data:
   From_Node  From_Layer  To_Node  To_Layer     Flow
0       18.0         1.0     22.0       1.0  0.00000
1       25.0         1.0     22.0       1.0  0.00000
2       64.0         1.0     26.0       1.0  0.00000
3       22.0         1.0     42.0       1.0  0.00000
4       19.0         1.0     60.0       1.0  2.28972


Layer 2 Data:
   From_Node  From_Layer  To_Node  To_Layer      Flow
4       22.0         2.0     25.0       2.0  0.000000
6       22.0         2.0     42.0       2.0  0.000000
7       22.0         2.0     18.0       2.0  0.000000
8       64.0         2.0     26.0       2.0  0.000000
9       19.0         2.0     60.0       2.0  2.310324


Layer 3 Data:
   From_Node  From_Layer  To_Node  To_Layer      Flow
0       42.0         3.0     22.0       3.0  0.000000
1       18.0         3.0     22.0       3.0  0.000000
2       64.0         3.0     26.0       3.0  0.000000
3       25.0         3.0     22.0       3.0  0.000000
5       19.0         3.0     60.0       3.

In [ ]:
#Cell #: 4 Multiplex Network Construction Logic
print("Initializing multiplex network...")
multiplex_net = ml.empty()

# A variable to keep track of the layer index
layer_index = 1

# Iterate over the filtered data and add each as a layer to the multiplex network
for df in filtered_cases_data:
    unique_layer_name = f"Layer_{layer_index}"
    print(f"\nAdding layer: {unique_layer_name}")

    # Debug: Print a sample of the DataFrame
    print(f"DataFrame sample from {unique_layer_name}:\n", df.head())

    # Prepare vertices and add them to the network
    vertices = {'actor': list(set(df['From_Node']).union(df['To_Node'])), 
                'layer': [unique_layer_name] * len(set(df['From_Node']).union(df['To_Node']))}
    print(f"Adding vertices to layer {unique_layer_name}: {vertices['actor']}")
    ml.add_vertices(multiplex_net, vertices)

    # Prepare edges and add them to the network
    edges = {
        'from_actor': df['From_Node'].tolist(),
        'from_layer': [unique_layer_name] * len(df),
        'to_actor': df['To_Node'].tolist(),
        'to_layer': [unique_layer_name] * len(df)
    }
    print(f"Adding {len(df)} edges to layer {unique_layer_name}")
    print(f"Sample edges for layer {unique_layer_name}: {edges['from_actor'][:5]} -> {edges['to_actor'][:5]}")
    ml.add_edges(multiplex_net, edges)

    layer_index += 1

print("\nNetwork construction completed.")
print(f"Total layers in network: {len(ml.layers(multiplex_net))}")

print("\nNetwork summary:")
print(ml.summary(multiplex_net))


In [ ]:
import itertools

def find_exclusive_neighbors_for_combination(multiplex_net, layers):
    exclusive_neighbors = {}
    for layer in layers:
        # Retrieve all vertices in the current layer
        vertices_info = ml.vertices(multiplex_net, [layer])
        actors = vertices_info['actor']  # List of actor IDs in the layer

        # Find exclusive neighbors for each actor in the layer
        for actor_id in actors:
            # Retrieve exclusive neighbors for the actor
            neighbors = ml.xneighbors(multiplex_net, actor_id, layers=[layer], mode='all')
            if actor_id in exclusive_neighbors:
                exclusive_neighbors[actor_id].update(neighbors)
            else:
                exclusive_neighbors[actor_id] = neighbors

    return exclusive_neighbors

# Assuming 'multiplex_net' is your multiplex network object
all_layers = ml.layers(multiplex_net)

# Step 1: Exclusive Neighbors for Individual Layers
exclusive_neighbors_individual_layers = {layer: find_exclusive_neighbors_for_combination(multiplex_net, [layer])
                                         for layer in all_layers}
print("Exclusive neighbors for individual layers:")
for layer, neighbors in exclusive_neighbors_individual_layers.items():
    print(f"Layer {layer}: {neighbors}")

# Step 2: Exclusive Neighbors for Two-Layer Combinations
two_layer_combinations = list(itertools.combinations(all_layers, 2))
exclusive_neighbors_two_layers = {comb: find_exclusive_neighbors_for_combination(multiplex_net, list(comb)) 
                                  for comb in two_layer_combinations}
print("\nExclusive neighbors for two-layer combinations:")
for comb, neighbors in exclusive_neighbors_two_layers.items():
    print(f"Combination {comb}: {neighbors}")

# Step 3: Exclusive Neighbors for Three-Layer Combinations
three_layer_combinations = list(itertools.combinations(all_layers, 3))
exclusive_neighbors_three_layers = {comb: find_exclusive_neighbors_for_combination(multiplex_net, list(comb)) 
                                    for comb in three_layer_combinations}
print("\nExclusive neighbors for three-layer combinations:")
for comb, neighbors in exclusive_neighbors_three_layers.items():
    print(f"Combination {comb}: {neighbors}")


In [ ]:
def modified_jaccard_coefficient(G, exclusive_neighbors):
    def predict(u, v):
        neighbors_u = exclusive_neighbors.get(u, set())
        neighbors_v = exclusive_neighbors.get(v, set())
        union_size = len(neighbors_u | neighbors_v)
        if union_size == 0:
            return 0
        intersection_size = len(neighbors_u & neighbors_v)
        return intersection_size / union_size

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

def calculate_jaccard_scores_for_layers(nx_graphs, layers, exclusive_neighbors):
    combined_graph = nx.Graph()
    for layer in layers:
        combined_graph = nx.compose(combined_graph, nx_graphs[layer])

    jaccard_scores = {}
    for u, v, score in modified_jaccard_coefficient(combined_graph, exclusive_neighbors):
        if score > 0:
            jaccard_scores[f"{u} - {v}"] = score
    return jaccard_scores

# Convert the multiplex network into NetworkX graphs for each layer
nx_graphs = ml.to_nx_dict(multiplex_net)

# Calculate Jaccard scores for individual layers
jaccard_scores_individual_layers = {layer: calculate_jaccard_scores_for_layers(nx_graphs, [layer], exclusive_neighbors_individual_layers[layer])
                                    for layer in all_layers}
print("Jaccard scores for individual layers:")
for layer, scores in jaccard_scores_individual_layers.items():
    print(f"Layer {layer}: {scores}")

# Calculate Jaccard scores for two-layer combinations
jaccard_scores_two_layers = {comb: calculate_jaccard_scores_for_layers(nx_graphs, list(comb), exclusive_neighbors_two_layers[comb])
                             for comb in two_layer_combinations}
print("\nJaccard scores for two-layer combinations:")
for comb, scores in jaccard_scores_two_layers.items():
    print(f"Combination {comb}: {scores}")

# Calculate Jaccard scores for three-layer combinations
jaccard_scores_three_layers = {comb: calculate_jaccard_scores_for_layers(nx_graphs, list(comb), exclusive_neighbors_three_layers[comb])
                               for comb in three_layer_combinations}
print("\nJaccard scores for three-layer combinations:")
for comb, scores in jaccard_scores_three_layers.items():
    print(f"Combination {comb}: {scores}")


In [ ]:
def modified_adamic_adar_index(G, exclusive_neighbors):
    def predict(u, v):
        common_neighbors = exclusive_neighbors.get(u, set()) & exclusive_neighbors.get(v, set())
        return sum(1 / log(len(exclusive_neighbors.get(w, set()))) for w in common_neighbors if len(exclusive_neighbors.get(w, set())) > 1)

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

def calculate_adamic_adar_scores_for_layers(nx_graphs, layers, exclusive_neighbors):
    combined_graph = nx.Graph()
    for layer in layers:
        combined_graph = nx.compose(combined_graph, nx_graphs[layer])

    adamic_adar_scores = {}
    for u, v, score in modified_adamic_adar_index(combined_graph, exclusive_neighbors):
        if score > 0:
            adamic_adar_scores[f"{u} - {v}"] = score
    return adamic_adar_scores

# Convert the multiplex network into NetworkX graphs for each layer
nx_graphs = ml.to_nx_dict(multiplex_net)

# Calculate Adamic-Adar scores for individual layers
adamic_adar_scores_individual_layers = {layer: calculate_adamic_adar_scores_for_layers(nx_graphs, [layer], exclusive_neighbors_individual_layers[layer])
                                        for layer in all_layers}
print("Adamic-Adar scores for individual layers:")
for layer, scores in adamic_adar_scores_individual_layers.items():
    print(f"Layer {layer}: {scores}")

# Calculate Adamic-Adar scores for two-layer combinations
adamic_adar_scores_two_layers = {comb: calculate_adamic_adar_scores_for_layers(nx_graphs, list(comb), exclusive_neighbors_two_layers[comb])
                                 for comb in two_layer_combinations}
print("\nAdamic-Adar scores for two-layer combinations:")
for comb, scores in adamic_adar_scores_two_layers.items():
    print(f"Combination {comb}: {scores}")

# Calculate Adamic-Adar scores for three-layer combinations
adamic_adar_scores_three_layers = {comb: calculate_adamic_adar_scores_for_layers(nx_graphs, list(comb), exclusive_neighbors_three_layers[comb])
                                   for comb in three_layer_combinations}
print("\nAdamic-Adar scores for three-layer combinations:")
for comb, scores in adamic_adar_scores_three_layers.items():
    print(f"Combination {comb}: {scores}")


In [ ]:
def normalize_by_max(scores):
    # Flatten all scores into a single list, excluding empty layers or combinations
    all_scores = [score for layer_scores in scores.values() for score in layer_scores.values() if layer_scores]

    # Check if there are no scores to normalize
    if not all_scores:
        print("No scores to normalize.")
        return {}

    # Find the maximum score for normalization
    max_score = max(all_scores)

    # Normalize the scores
    normalized_scores = {}
    for key, layer_scores in scores.items():
        if layer_scores:  # Check if the layer or combination has scores
            normalized_scores[key] = {link: score / max_score for link, score in layer_scores.items()}
    
    return normalized_scores

# Normalize Jaccard and Adamic-Adar scores for individual layers, two-layer combinations, and three-layer combinations
normalized_jaccard_scores_individual_layers = normalize_by_max(jaccard_scores_individual_layers)
normalized_jaccard_scores_two_layers = normalize_by_max(jaccard_scores_two_layers)
normalized_jaccard_scores_three_layers = normalize_by_max(jaccard_scores_three_layers)

normalized_adamic_adar_scores_individual_layers = normalize_by_max(adamic_adar_scores_individual_layers)
normalized_adamic_adar_scores_two_layers = normalize_by_max(adamic_adar_scores_two_layers)
normalized_adamic_adar_scores_three_layers = normalize_by_max(adamic_adar_scores_three_layers)

# Display the normalized scores
print("Normalized Jaccard Scores for Individual Layers:")
for layer, scores in normalized_jaccard_scores_individual_layers.items():
    print(f"{layer}: {scores}")

print("\nNormalized Jaccard Scores for Two-Layer Combinations:")
for combination, scores in normalized_jaccard_scores_two_layers.items():
    print(f"{combination}: {scores}")

print("\nNormalized Jaccard Scores for Three-Layer Combinations:")
for combination, scores in normalized_jaccard_scores_three_layers.items():
    print(f"{combination}: {scores}")

print("\nNormalized Adamic-Adar Scores for Individual Layers:")
for layer, scores in normalized_adamic_adar_scores_individual_layers.items():
    print(f"{layer}: {scores}")

print("\nNormalized Adamic-Adar Scores for Two-Layer Combinations:")
for combination, scores in normalized_adamic_adar_scores_two_layers.items():
    print(f"{combination}: {scores}")

print("\nNormalized Adamic-Adar Scores for Three-Layer Combinations:")
for combination, scores in normalized_adamic_adar_scores_three_layers.items():
    print(f"{combination}: {scores}")


In [ ]:
def scores_to_dataframe(normalized_scores, layer_type):
    links = []
    for layer, layer_scores in normalized_scores.items():
        for link, score in layer_scores.items():
            node_u, node_v = map(float, link.split(' - '))
            if layer_type == "individual":
                layer_label = layer
            else:
                # Format the layer label correctly for combinations
                layer_label = tuple(layer)
            links.append({'Node_U': node_u, 'Node_V': node_v, 'Probability': score, 'Layer': layer_label})
    return pd.DataFrame(links)

# Convert normalized scores to DataFrames for individual layers, two-layer combinations, and three-layer combinations
df_normalized_jaccard_individual = scores_to_dataframe(normalized_jaccard_scores_individual_layers, "individual")
df_normalized_jaccard_two_layers = scores_to_dataframe(normalized_jaccard_scores_two_layers, "combination")
df_normalized_jaccard_three_layers = scores_to_dataframe(normalized_jaccard_scores_three_layers, "combination")

df_normalized_adamic_adar_individual = scores_to_dataframe(normalized_adamic_adar_scores_individual_layers, "individual")
df_normalized_adamic_adar_two_layers = scores_to_dataframe(normalized_adamic_adar_scores_two_layers, "combination")
df_normalized_adamic_adar_three_layers = scores_to_dataframe(normalized_adamic_adar_scores_three_layers, "combination")
# Display the first few rows of each DataFrame
print("DataFrame with Normalized Jaccard Scores for Individual Layers:")
print(df_normalized_jaccard_individual.head())

print("\nDataFrame with Normalized Jaccard Scores for Two-Layer Combinations:")
print(df_normalized_jaccard_two_layers.head())

print("\nDataFrame with Normalized Jaccard Scores for Three-Layer Combinations:")
print(df_normalized_jaccard_three_layers.head())

print("\nDataFrame with Normalized Adamic-Adar Scores for Individual Layers:")
print(df_normalized_adamic_adar_individual.head())

print("\nDataFrame with Normalized Adamic-Adar Scores for Two-Layer Combinations:")
print(df_normalized_adamic_adar_two_layers.head())

print("\nDataFrame with Normalized Adamic-Adar Scores for Three-Layer Combinations:")
print(df_normalized_adamic_adar_three_layers.head())


In [ ]:
def calculate_average_flow(node, exclusive_neighbors, layer_data):
    total_flow = 0
    node_str = str(node)  # Convert node ID to string for key lookup

    # Check if the node has exclusive neighbors in the given layer
    if node_str not in exclusive_neighbors:
        return 0  # Return 0 if node has no exclusive neighbors in this layer

    neighbor_count = len(exclusive_neighbors[node_str])
    for neighbor_str in exclusive_neighbors[node_str]:
        neighbor = float(neighbor_str)  # Convert neighbor ID back to float for comparison

        # Extract flow data for edges between the node and its neighbors
        flow = layer_data.loc[((layer_data['From_Node'] == node) & (layer_data['To_Node'] == neighbor)) |
                              ((layer_data['From_Node'] == neighbor) & (layer_data['To_Node'] == node)), 'Flow']

        if not flow.empty:
            total_flow += flow.iloc[0]  # Add the flow value to the total

    # Calculate average flow if there are neighbors, otherwise return 0
    return total_flow / neighbor_count if neighbor_count > 0 else 0

def calculate_edge_weight(row, exclusive_neighbors, filtered_cases_data, layer_type):
    if layer_type == "individual":
        layer = row['Layer']
        layer_index = int(layer.split('_')[1]) - 1
        layer_data = filtered_cases_data[layer_index]
        final_avg_flow_u = calculate_average_flow(row['Node_U'], exclusive_neighbors, layer_data)
        final_avg_flow_v = calculate_average_flow(row['Node_V'], exclusive_neighbors, layer_data)
    else:
        # Handle layer combinations directly as a tuple
        layers = row['Layer']
        avg_flows_u = []
        avg_flows_v = []

        for layer in layers:
            layer_index = int(layer.split('_')[1]) - 1
            layer_data = filtered_cases_data[layer_index]

            avg_flow_u = calculate_average_flow(row['Node_U'], exclusive_neighbors, layer_data)
            avg_flow_v = calculate_average_flow(row['Node_V'], exclusive_neighbors, layer_data)

            if avg_flow_u is not None:
                avg_flows_u.append(avg_flow_u)
            if avg_flow_v is not None:
                avg_flows_v.append(avg_flow_v)

        final_avg_flow_u = sum(avg_flows_u) / len(avg_flows_u) if avg_flows_u else 0
        final_avg_flow_v = sum(avg_flows_v) / len(avg_flows_v) if avg_flows_v else 0

    final_weight = (final_avg_flow_u + final_avg_flow_v) / 2 * row['Probability']
    return final_weight

def find_node_layer(node, exclusive_neighbors):
    # Adjust the logic to identify the layer(s) a node belongs to
    layers = []
    for layer, neighbors in exclusive_neighbors.items():
        if str(node) in neighbors:
            layers.append(layer)
    return layers


In [ ]:
# How to apply the calculate_edge_weight function to the DataFrames
# Assuming df_normalized_jaccard_individual is your DataFrame for individual layers
#df_normalized_jaccard_individual['weight'] = df_normalized_jaccard_individual.apply(
 #   lambda row: calculate_edge_weight(row, 
 #                                     exclusive_neighbors_individual_layers[row['Layer']], 
 #                                     filtered_cases_data, 
 #                                     "individual"),
  #  axis=1
#)
# Now apply the calculate_edge_weight function
#df_normalized_jaccard_three_layers['weight'] = df_normalized_jaccard_three_layers.apply(
 #   lambda row: calculate_edge_weight(row,
 #                                     exclusive_neighbors_three_layers[row['Layer']],
  #                                    filtered_cases_data,
  #                                    "combination"),
   # axis=1
#)
#print("\nDataFrame with Normalized Jaccard Scores for Three-Layer Combinations:")
#print(df_normalized_jaccard_three_layers.head())

In [ ]:
def process_and_format_df(df, exclusive_neighbors, filtered_cases_data, layer_type):
    # Apply weight calculation
    df['weight'] = df.apply(
        lambda row: calculate_edge_weight(row, exclusive_neighbors[row['Layer']], filtered_cases_data, layer_type),
        axis=1
    )

    # Format for export
    formatted_df = df[['Node_U', 'Node_V', 'Layer', 'weight']].copy()
    if layer_type != "individual":
        # For layer combinations, we need to add this link to all layers in the combination
        combined_dfs = []
        for index, row in formatted_df.iterrows():
            for layer in row['Layer']:
                combined_dfs.append({'Node_U': row['Node_U'], 'Node_V': row['Node_V'], 'Layer': layer, 'weight': row['weight']})
        formatted_df = pd.DataFrame(combined_dfs)
    
    return formatted_df

# Apply the function to all the DataFrames
df_jaccard_individual_processed = process_and_format_df(df_normalized_jaccard_individual, exclusive_neighbors_individual_layers, filtered_cases_data, "individual")
df_jaccard_two_layers_processed = process_and_format_df(df_normalized_jaccard_two_layers, exclusive_neighbors_two_layers, filtered_cases_data, "combination")
df_jaccard_three_layers_processed = process_and_format_df(df_normalized_jaccard_three_layers, exclusive_neighbors_three_layers, filtered_cases_data, "combination")

df_adamic_adar_individual_processed = process_and_format_df(df_normalized_adamic_adar_individual, exclusive_neighbors_individual_layers, filtered_cases_data, "individual")
df_adamic_adar_two_layers_processed = process_and_format_df(df_normalized_adamic_adar_two_layers, exclusive_neighbors_two_layers, filtered_cases_data, "combination")
df_adamic_adar_three_layers_processed = process_and_format_df(df_normalized_adamic_adar_three_layers, exclusive_neighbors_three_layers, filtered_cases_data, "combination")

# Combine all processed DataFrames
combined_df = pd.concat([
    df_jaccard_individual_processed,
    df_jaccard_two_layers_processed,
    df_jaccard_three_layers_processed,
    df_adamic_adar_individual_processed,
    df_adamic_adar_two_layers_processed,
    df_adamic_adar_three_layers_processed
])

# Export the combined DataFrame to a CSV file
#combined_df.to_csv('/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/processed_network_data.csv', index=False)

def export_processed_data(df, exclusive_neighbors, filtered_cases_data, layer_type, filename_prefix):
    processed_df = process_and_format_df(df, exclusive_neighbors, filtered_cases_data, layer_type)
    file_path = f'/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/{filename_prefix}_{layer_type}.csv'
    processed_df.to_csv(file_path, index=False)
    return file_path

# Exporting each DataFrame to a separate CSV file
file_jaccard_individual = export_processed_data(df_normalized_jaccard_individual, exclusive_neighbors_individual_layers, filtered_cases_data, "individual", "jaccard")
file_jaccard_two_layers = export_processed_data(df_normalized_jaccard_two_layers, exclusive_neighbors_two_layers, filtered_cases_data, "two_layers", "jaccard")
file_jaccard_three_layers = export_processed_data(df_normalized_jaccard_three_layers, exclusive_neighbors_three_layers, filtered_cases_data, "three_layers", "jaccard")

file_adamic_adar_individual = export_processed_data(df_normalized_adamic_adar_individual, exclusive_neighbors_individual_layers, filtered_cases_data, "individual", "adamic_adar")
file_adamic_adar_two_layers = export_processed_data(df_normalized_adamic_adar_two_layers, exclusive_neighbors_two_layers, filtered_cases_data, "two_layers", "adamic_adar")
file_adamic_adar_three_layers = export_processed_data(df_normalized_adamic_adar_three_layers, exclusive_neighbors_three_layers, filtered_cases_data, "three_layers", "adamic_adar")

# Returning the file paths for download
[file_jaccard_individual, file_jaccard_two_layers, file_jaccard_three_layers, 
 file_adamic_adar_individual, file_adamic_adar_two_layers, file_adamic_adar_three_layers]
